AInewsbot.ipynb

- Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
- Save HTML of each URL in htmldata directory
- Extract URLs from all files, create a pandas dataframe with url, title, src
- Use ChatGPT to filter only AI-related headlines by sending a prompt and formatted table of headlines
- Use SQLite to filter headlines previously seen 
- OPENAI_API_KEY should be in the environment or in a .env file
  
Alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


In [1]:
import json
import os
import re
from datetime import datetime, timedelta
from urllib.parse import urlparse
import time
import yaml

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import bs4
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

import openai
from openai import OpenAI
import tiktoken

import dotenv

import sqlite3

import IPython
from IPython.display import HTML, Markdown, display

from atproto import Client

import PIL
from PIL import Image

print(f"openai          {openai.__version__}")
print(f"requests        {requests.__version__}")
print(f"BeautifulSoup   {bs4.__version__}")

openai          1.16.2
requests        2.31.0
BeautifulSoup   4.12.3


In [2]:
print(datetime.now())

2024-04-25 13:40:03.075820


In [3]:
# load credentials if necessary
dotenv.load_dotenv()
client = OpenAI()

In [4]:
# delete files in output directory
download_dir = "htmldata"

def delete_files(outputdir):

    # Iterate over all files in the directory
    for filename in os.listdir(outputdir):
        if filename.startswith('.'):
            continue
        file_path = os.path.join(outputdir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                # If you want to remove subdirectories as well, use os.rmdir() here
                pass
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

In [5]:
delete_files(download_dir)

# Specify sources

In [6]:
# load sources from YAML file
with open("sources.yaml", "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

sources_reverse = {d['title']:src for src, d in sources.items()}

sources

{'Ars Technica': {'include': ['^https://arstechnica.com/gadgets/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/'},
 'Bloomberg Tech': {'include': ['^https://www.bloomberg.com/news/(\\w+)/(\\d+)-(\\d+)-(\\d+)'],
  'title': 'Bloomberg Technology - Bloomberg',
  'url': 'https://www.bloomberg.com/technology'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology'},
 'Feedly AI': {'scroll': 2,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY

# Download HTML files from sources

In [7]:
# download files via selenium and firefox
outputdir = "htmldata"
delete_files(outputdir)

# Print the formatted time
print(datetime.now().strftime('%H:%M:%S'), "Starting", flush=True)

firefox_app_path = '/Applications/Firefox.app'
# Path to your geckodriver
geckodriver_path = '/Users/drucev/webdrivers/geckodriver'

# Set up Firefox options to use your existing profile
# important for some sites that need a login, also a generic profile fingerprint that looks like a bot might get blocked
firefox_profile_path = '/Users/drucev/Library/Application Support/Firefox/Profiles/k8k0lcjj.default-release'
options = Options()
options.profile = firefox_profile_path

print(datetime.now().strftime('%H:%M:%S'), "Initialized profile", flush=True)

# Create a Service object with the path
service = Service(geckodriver_path)

print(datetime.now().strftime('%H:%M:%S'), "Initialized service", flush=True)
# Set up the Firefox driver
driver = webdriver.Firefox(service=service, options=options)

print(datetime.now().strftime('%H:%M:%S'), "Initialized webdriver", flush=True)
sleeptime = 10

for sourcename, sourcedict in sources.items():
    print(datetime.now().strftime('%H:%M:%S'), f'Processing {sourcename}', flush=True)
    title = sourcedict["title"]
    url = sourcedict["url"]
    scroll = sourcedict.get("scroll", 0)
    click = sourcedict.get("click")

    # Open the page
    driver.get(url)

    # Wait for the page to load
    time.sleep(sleeptime)  # Adjust the sleep time as necessary

    if click:
        print(datetime.now().strftime('%H:%M:%S'), f"Clicking on {click}", flush=True)
        button = driver.find_element(By.XPATH, click)
        if button:
            button.click()
            print(datetime.now().strftime('%H:%M:%S'), f"Clicked", flush=True)

    for _ in range(scroll):
        # scroll to bottom of infinite scrolling window
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print(datetime.now().strftime('%H:%M:%S'), "Loading additional infinite scroll items", flush=True)
        time.sleep(sleeptime) # wait for it to load additional items

    # Get the HTML source of the page
    html_source = driver.page_source

    # check encoding, default utf-8
    encoding = "utf-8"  # Default to UTF-8 if not specified
    # Retrieve the content-type meta tag from the HTML
    try:
        meta_tag = driver.find_element(By.XPATH, "//meta[@http-equiv='Content-Type']")
        content_type = meta_tag.get_attribute("content")
        # Typical format is "text/html; charset=UTF-8"
        charset_start = content_type.find("charset=")
        if charset_start != -1:
            encoding = content_type[charset_start + 8:]
    except Exception as err:
        pass

    # Save the HTML to a local file
    datestr = datetime.now().strftime("%m_%d_%Y %I_%M_%S %p")
    outfile = f'{title} ({datestr}).html'
    print(datetime.now().strftime('%H:%M:%S'), f"Saving {outfile} as {encoding}", flush=True)
    with open(outputdir + "/" + outfile, 'w', encoding=encoding) as file:
        file.write(html_source)

# Close the browser
driver.quit()
print(datetime.now().strftime('%H:%M:%S'), "Quit webdriver", flush=True)


13:40:19 Starting
13:40:30 Initialized profile
13:40:30 Initialized service
13:41:11 Initialized webdriver
13:41:11 Processing Ars Technica
13:41:22 Saving Ars Technica (04_25_2024 01_41_22 PM).html as utf-8
13:41:22 Processing Bloomberg Tech
13:41:33 Saving Bloomberg Technology - Bloomberg (04_25_2024 01_41_33 PM).html as utf-8
13:41:33 Processing Business Insider
13:41:44 Saving Tech - Business Insider (04_25_2024 01_41_44 PM).html as utf-8
13:41:44 Processing FT Tech
13:41:54 Saving Technology (04_25_2024 01_41_54 PM).html as utf-8
13:41:54 Processing Feedly AI
13:42:05 Loading additional infinite scroll items
13:42:15 Loading additional infinite scroll items
13:42:25 Saving Discover and Add New Feedly AI Feeds (04_25_2024 01_42_25 PM).html as utf-8
13:42:25 Processing Google News
13:42:36 Clicking on //*[@aria-label="Artificial intelligence"]
13:42:37 Clicked
13:42:37 Loading additional infinite scroll items
13:42:47 Loading additional infinite scroll items
13:42:57 Saving Google N

In [8]:
[os.path.join(download_dir, file) for file in os.listdir(download_dir)]

['htmldata/Ars Technica (04_25_2024 01_41_22 PM).html',
 'htmldata/AI News _ VentureBeat (04_25_2024 01_44_55 PM).html',
 'htmldata/HackerNoon - read, write and learn about any technology (04_25_2024 01_43_29 PM).html',
 'htmldata/Technology (04_25_2024 01_41_54 PM).html',
 'htmldata/Hacker News Page 2 (04_25_2024 01_43_18 PM).html',
 'htmldata/Bloomberg Technology - Bloomberg (04_25_2024 01_41_33 PM).html',
 'htmldata/Google News - Technology - Artificial intelligence (04_25_2024 01_42_57 PM).html',
 'htmldata/Techmeme (04_25_2024 01_44_24 PM).html',
 'htmldata/Hacker News Page 1 (04_25_2024 01_43_07 PM).html',
 'htmldata/.gitkeep',
 'htmldata/top scoring links _ multi (04_25_2024 01_44_13 PM).html',
 'htmldata/Technology - The Washington Post (04_25_2024 01_45_16 PM).html',
 'htmldata/Artificial Intelligence - The Verge (04_25_2024 01_44_45 PM).html',
 'htmldata/The Register_ Enterprise Technology News and Analysis (04_25_2024 01_44_34 PM).html',
 'htmldata/Discover and Add New Feedl

In [9]:
# List all paths in the directory matching today's date
nfiles = 50

# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day

datestr = datetime.now().strftime("%m_%d_%Y")

# print(f"Year: {year}, Month: {month}, Day: {day}")

files = [os.path.join(download_dir, file) for file in os.listdir(download_dir)]
# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [file for file in files if datestr in file and file.endswith(".html")]
print(len(files))
for file in files:
    print(file)

17
htmldata/Technology - The Washington Post (04_25_2024 01_45_16 PM).html
htmldata/Technology - WSJ.com (04_25_2024 01_45_06 PM).html
htmldata/AI News _ VentureBeat (04_25_2024 01_44_55 PM).html
htmldata/Artificial Intelligence - The Verge (04_25_2024 01_44_45 PM).html
htmldata/The Register_ Enterprise Technology News and Analysis (04_25_2024 01_44_34 PM).html
htmldata/Techmeme (04_25_2024 01_44_24 PM).html
htmldata/top scoring links _ multi (04_25_2024 01_44_13 PM).html
htmldata/Technology - The New York Times (04_25_2024 01_43_40 PM).html
htmldata/HackerNoon - read, write and learn about any technology (04_25_2024 01_43_29 PM).html
htmldata/Hacker News Page 2 (04_25_2024 01_43_18 PM).html
htmldata/Hacker News Page 1 (04_25_2024 01_43_07 PM).html
htmldata/Google News - Technology - Artificial intelligence (04_25_2024 01_42_57 PM).html
htmldata/Discover and Add New Feedly AI Feeds (04_25_2024 01_42_25 PM).html
htmldata/Technology (04_25_2024 01_41_54 PM).html
htmldata/Tech - Business 

In [10]:
# you need this if you have not-descriptive link titles like 'link', can get a page title from html or tags
def get_og_tags(url):
    """get a dict of Open Graph og: tags such as title in the HEAD of a URL"""
    retdict = {}
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            head = soup.head
            if head:
                og_tags = head.find_all(
                    property=lambda prop: prop and prop.startswith("og:")
                )
                for tag in og_tags:
                    if "content" in tag.attrs:
                        retdict[tag["property"]] = tag["content"]

                page_title = ""
                title_tag = soup.find("title")
                if title_tag:
                    page_title = title_tag.text
                    if page_title:
                        retdict["title"] = page_title
        return retdict
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
    return retdict


url = "https://www.euronews.com/next/2024/01/15/almost-40-of-jobs-around-the-world-will-be-impacted-by-ai-imf-chief-says"
get_og_tags(url)

{'og:locale': 'en-GB',
 'og:url': 'https://www.euronews.com/next/2024/01/15/almost-40-of-jobs-around-the-world-will-be-impacted-by-ai-imf-chief-says',
 'og:site_name': 'euronews',
 'og:type': 'article',
 'og:title': 'AI to impact 40% of jobs around the world, IMF chief says',
 'og:description': 'Kristalina Georgieva said now is the time to act to create a set of policies ensuring the impact of AI is beneficial not detrimental to humanity.',
 'og:image': 'https://static.euronews.com/articles/stories/08/17/12/08/1200x675_cmsv2_1bac2582-b418-5da9-80f9-6c4b6254606d-8171208.jpg',
 'og:image:width': '1200',
 'og:image:height': '675',
 'og:image:type': 'image/jpeg',
 'og:image:alt': 'Almost 40% of jobs around the world will be impacted by AI, IMF chief says',
 'og:locale:alternate': 'el-GR',
 'og:locale:alternate:url': 'https://www.euronews.com/next/2024/01/15/almost-40-of-jobs-around-the-world-will-be-impacted-by-ai-imf-chief-says',
 'title': 'Almost 40% of jobs around the world will be impa

In [11]:
def get_path_from_url(url):
    """
    Extracts the path following the top-level domain name from a URL.

    :param url: The URL string.
    :return: The path component of the URL.
    """
    parsed_url = urlparse(url)
    return parsed_url.path


# Example usage
example_url = "http://www.example.com/some/path?query=string"
path = get_path_from_url(example_url)
print(path)

/some/path


In [12]:
# https://platform.openai.com/docs/models/gpt-4-turbo-and-gpt-4
MODEL = "gpt-4-turbo"

MAX_INPUT_TOKENS = 3072
MAX_OUTPUT_TOKENS = 4096
MAX_RETRIES = 3
TEMPERATURE = 0

# models = sorted(openai.models.list(), key=lambda m: m.created)
# models

In [13]:
enc = tiktoken.encoding_for_model(MODEL)
assert enc.decode(enc.encode("hello world")) == "hello world"


def count_tokens(s):
    return len(enc.encode(s))


count_tokens("four score and 7 years go our forefathers brought forth")

13

In [14]:
def trimmed_href(l):
    """
    Trims everything in the string after a question mark such as a session ID.

    :param s: The input string.
    :return: The trimmed string.
    """
    # Find the position of the question mark
    s = l.get("href")
    if s:
        question_mark_index = s.find("?")

        # If a question mark is found, trim the string up to that point
        if question_mark_index != -1:
            return s[:question_mark_index]
        else:
            # Return the original string if no question mark is found
            return s
    else:
        return s

# Parse news URLs and titles from downloaded HTML files

In [15]:
# parse all the URL that look like news articles
# into all_urls list of dicts with url, title, src
all_urls = []

for file in files:
    # Extract filename from path
    filename = os.path.basename(file)

    # Find the position of '1_14_2024' in the filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
#     print(basename)
#     if basename.startswith('Google News'):
#         pass
#     else:
#         continue

    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        print(f"Skipping {basename}, no sourcename metadata")
        continue

    display(Markdown(f"# {sourcename}"))
    sources[sourcename]["latest"] = file

    # get contents
    with open(file, "r") as file:
        html_content = file.read()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Find all <a> tags
    if soup:
        links = soup.find_all("a")
    else:
        print(f"Skipping {sourcename}, unable to parse")

    # convert relative links to absolute links using base URL if present
    base_tag = soup.find('base')
    base_url = base_tag.get('href') if base_tag else sources[sourcename]["url"]
    for link in links:
#         print(link.get("href"))
        link["href"]= urljoin(base_url, link.get('href', ""))
#         print(link["href"])

#     print(len(links))
#     links = [l for l in links if l]
#     links = [l.strip() for l in links]

    print(len(links))

    for pattern in sources[sourcename].get("exclude", []):
        # filter links by exclusion pattern
#         print(pattern)
#         print([ l.get("href") for l in links])
        links = [
            l
            for l in links
            if l.get("href") is not None and not re.match(pattern, l.get("href"))
        ]
        # print(len(links))

    for pattern in sources[sourcename].get("include", []):
        # print(pattern, len(links))
        # filter links by inclusion pattern
        # print(pattern)
        # print(type(pattern))
        newlinks = []
        for l in links:
            href = l.get("href")
#             print(href)
            if href and re.match(pattern, href):
                newlinks.append(l)
        links = newlinks
        # links = [l for l in links if re.match(pattern, l.get("href"))]
        # print(len(links))

    # drop empty text
    links = [l for l in links if l.get_text(strip=True)]

    # drop empty url path, i.e. url = toplevel domain
    links = [l for l in links if len(get_path_from_url(trimmed_href(l))) > 1]
    # drop anything that is not http, like javascript: or mailto:
    links = [l for l in links if l.get("href") and l.get("href").startswith("http")]
    # drop some ArsTechnica links that are just the number of comments and dupe the primary link
    links = [l for l in links if not re.match("^(\d+)$", l.get_text(strip=True))]

    for l in links:
        url = trimmed_href(l)
        title = l.get_text(strip=True)
        if title == "LINK":
            # try to update title
            og_dict = get_og_tags(url)
            if og_dict.get("og:title"):
                title = og_dict.get("og:title")

        # skip some low quality links that don't have full headline, like link to a Twitter or Threads account
        if len(title) <= 28 and title != "LINK":
            continue

        all_urls.append({"title": title, "url": url, "src": sourcename})
#         display(Markdown(f"[{title}]({url})"))

    print(len(links))
    print()

    # for p in pages:
    #     print(p)

# WaPo Tech

164
38



# WSJ Tech

545
14



# VentureBeat

...

339
253



# Bloomberg Tech

303
52



# Ars Technica

252
8



In [16]:
# make a pandas dataframe
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
orig_df

,id,src,title,url
0,0,Ars Technica,A Polestar Phone now inexplicably exists,https://arstechnica.com/gadgets/2024/04/a-pole...
1,1,Ars Technica,Google can’t quit third-party cookies—delays s...,https://arstechnica.com/gadgets/2024/04/google...
2,2,Ars Technica,Is the Arm version of Windows ready for its cl...,https://arstechnica.com/gadgets/2024/04/is-the...
3,3,Ars Technica,Qualcomm says lower-end Snapdragon X Plus chip...,https://arstechnica.com/gadgets/2024/04/qualco...
4,4,Ars Technica,The spam came from inside the house: How a sma...,https://arstechnica.com/gadgets/2024/04/the-sp...
...,...,...,...,...
1075,1075,WaPo Tech,"FCC reinstates net neutrality, but it’s not as...",https://www.washingtonpost.com/technology/2024...
1076,1076,WaPo Tech,Trump got one thing right: Banning TikTok woul...,https://www.washingtonpost.com/technology/2024...
1077,1077,WaPo Tech,The U.S. could ban TikTok. These countries hav...,https://www.washingtonpost.com/world/2024/03/1...
1078,1078,WaPo Tech,Google fires more workers after CEO says workp...,https://www.washingtonpost.com/technology/2024...


In [17]:
# filter ones not seen before
conn = sqlite3.connect('articles.db')

# Retrieve all URLs from the SQLite table
existing_urls = pd.read_sql_query("SELECT url FROM news_articles", conn)

# Close the SQLite connection
conn.close()

# Convert the URLs to a list for easier comparison
existing_urls_list = existing_urls['url'].tolist()

# Filter the original DataFrame
# Keep rows where the URL is not in the existing_urls_list
filtered_df = orig_df[~orig_df['url'].isin(existing_urls_list)]


In [18]:
len(existing_urls_list)

51977

In [19]:
len(filtered_df)

365

# Filter AI-related headlines using a prompt to OpenAI

In [20]:
# make pages that fit in a reasonably sized prompt
MAXPAGELEN = 50
pages = []
current_page = []
pagelength = 0

for row in filtered_df.itertuples():
    curlink = {"id": row.Index, "title": row.title}
    curlength = count_tokens(json.dumps(curlink))
    # Check if adding the current string would exceed the limit
    if len(current_page) >= MAXPAGELEN or pagelength + curlength > MAX_INPUT_TOKENS:
        # If so, start a new page
        pages.append(current_page)
        current_page = [curlink]
        pagelength = curlength
    else:
        # Otherwise, add the string to the current page
        current_page.append(curlink)
        pagelength += curlength

# add the last page if it's not empty
if current_page:
    pages.append(current_page)

len(pages)

8

In [21]:
def get_response_json(
    client,
    messages,
    verbose=False,
    model=MODEL,
    # max_input_tokens=MAX_INPUT_TOKENS,
    max_output_tokens=MAX_OUTPUT_TOKENS,
    max_retries=MAX_RETRIES,
    temperature=TEMPERATURE,
):
    if type(messages) != list:  # allow passing one string for convenience
        messages = [{"role": "user", "content": messages}]

    if verbose:
        print("\n".join([str(msg) for msg in messages]))

    # truncate number of tokens
    # retry loop, have received untrapped 500 errors like too busy
    for i in range(max_retries):
        if i > 0:
            print(f"Attempt {i+1}...")
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                temperature=0,
                max_tokens=max_output_tokens,
                response_format={"type": "json_object"},
            )
            # no exception thrown
            return response
        except Exception as error:
            print(f"An exception occurred on attempt {i+1}:", error)
            time.sleep(5)
            continue  # try again
        # retries exceeded if you got this far
    print("Retries exceeded.")
    return None


# messages = [
#     {
#         "role": "system",
#         "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
#     },
#     {
#         "role": "user",
#         "content": "Compose a poem that explains the concept of recursion in programming, returning each verse as a json object.",
#     },
# ]

# response = get_response_json(client, messages)
# response

In [22]:
prompt = """
You will act as a research assistant classifying news stories as related to artificial intelligence (AI) or unrelated to AI.

Your task is to read JSON format objects from an input list of news stories using the schema below delimited by |, and output JSON format objects for each using the schema below delimited by ~.

Define a list of objects representing news stories in JSON format as in the following example:
|
{'stories':
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
 ]
}
|

Based on the title, you will classify each story as being about AI or not.

For each object, you will output the input id field, and a field named isAI which is true if the input title is about AI and false if the input title is not about AI.

When extracting information please make sure it matches the JSON format below exactly. Do not output any attributes that do not appear in the schema below.
~
{'stories':
[{'id': 97, 'isAI': true},
 {'id': 103, 'isAI': true},
 {'id': 103, 'isAI': false},
 {'id': 210, 'isAI': true},
 {'id': 298, 'isAI': false}]
}
~

You may interpret the term AI broadly as pertaining to
- machine learning models
- large language models
- robotics
- reinforcement learning
- computer vision
- OpenAI
- ChatGPT
- other closely related topics.

You will return an array of valid JSON objects.

The field 'id' in the output must match the field 'id' in the input EXACTLY.

The field 'isAI' must be either true or false.

The list of news stories to classify and enrich is:


"""

In [23]:
pages[0][0]

{'id': 3,
 'title': 'Qualcomm says lower-end Snapdragon X Plus chips can still outrun Apple’s M3'}

In [24]:
def fetch_response(client, prompt, p):
    """given a dict p (page) of keys and values,
       prompt the openai client to process the dict 
       and return the response as a list of keys and values """
    response = get_response_json(client, prompt + json.dumps(p))
    responses.append(response.choices[0].message.content)
    retval = json.loads(responses[-1])
    retlist = []
    # usually comes back as a dict with a single arbitrary key like "stories" with a list value
    if type(retval) == dict:
        for k, v in retval.items():
            if type(v) == list:
                retlist.extend(v)
            else:
                retlist.append(v)
        print(
            f"{datetime.now().strftime('%H:%M:%S')} got dict with {len(retlist)} items "
        )
    elif type(retval) == list:  # in case it comes back as a list
        retlist = retval
        print(
            f"{datetime.now().strftime('%H:%M:%S')} got list with {len(retlist)} items "
        )
    else:
        print(str(type(retval)))
    # make a list of ids sent
    sent_ids = [ s['id'] for s in p]    
    # list of ids got back
    received_ids = [r['id'] for r in retval['stories']]
    # subtract response from sent, check if empty
    difference = set(sent_ids) - set(received_ids)
    # could map response to boolean, remove anything from response that is not true or false
    if difference:
        print("missing items", difference)
        return []
    else:
        return retlist
        
responses = []
enriched_urls = []
for i, p in enumerate(pages):
    print(
        f"{datetime.now().strftime('%H:%M:%S')} send page {i+1} of {len(pages)}, {len(p)} items "
    )
    # print(prompt + json.dumps(p))
    for c in range(3):
        if c:
            print(f"Retrying, attempt {c+1}")
        retlist = fetch_response(client, prompt, p)
        if retlist:
            break
    if retlist:
        enriched_urls.extend(retlist)
    else:
        print(f"Failed after {c+1} attempts")


14:00:58 send page 1 of 8, 50 items 
14:01:32 got dict with 50 items 
14:01:32 send page 2 of 8, 50 items 
14:02:11 got dict with 50 items 
14:02:11 send page 3 of 8, 50 items 
14:02:53 got dict with 50 items 
14:02:53 send page 4 of 8, 50 items 
14:03:39 got dict with 50 items 
14:03:39 send page 5 of 8, 50 items 
14:04:15 got dict with 50 items 
14:04:15 send page 6 of 8, 50 items 
14:04:50 got dict with 50 items 
14:04:50 send page 7 of 8, 50 items 
14:05:29 got dict with 50 items 
14:05:29 send page 8 of 8, 15 items 
14:05:40 got dict with 15 items 


In [25]:
enriched_df = pd.DataFrame(enriched_urls)
enriched_df.head()


,id,isAI
0,3,False
1,10,False
2,21,False
3,23,False
4,56,True


In [26]:
print("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
print("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))

isAI 156
not isAI 209


In [27]:
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date']=datetime.now().date()
merged_df.head()

,id,src,title,url,isAI,date
0,3,Ars Technica,Qualcomm says lower-end Snapdragon X Plus chip...,https://arstechnica.com/gadgets/2024/04/qualco...,False,2024-04-25
1,10,Bloomberg Tech,‘Alien' Joins Old Movie Hits Returning to Thea...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-04-25
2,21,Bloomberg Tech,Microsoft-Backed Rubrik Shares Jump 25% After ...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-04-25
3,23,Bloomberg Tech,US Revives Net Neutrality to Establish Authori...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-04-25
4,56,Business Insider,How Hitachi Digital Services is helping compan...,https://www.businessinsider.com/sc/how-compani...,True,2024-04-25


In [28]:
# ideally should be empty, shouldn't get back rows that don't match to existing
merged_df.loc[merged_df["src"].isna()]

,id,src,title,url,isAI,date


In [29]:
# ideally should be empty, should get back all rows from orig
merged_df.loc[merged_df["isAI"].isna()]

,id,src,title,url,isAI,date


In [30]:
# # Connect to SQLite database
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

# # Create table with a date column
# cursor.execute('''
# CREATE TABLE IF NOT EXISTS news_articles (
#     id INTEGER PRIMARY KEY,
#     src TEXT,
#     title TEXT,
#     url TEXT UNIQUE,
#     isAI BOOLEAN,
#     article_date DATE
# )
# ''')
# conn.commit()
# conn.close()

In [31]:
# Function to insert a new article
def insert_article(cursor, src, title, url, isAI, article_date):
    try:
        cursor.execute("INSERT OR IGNORE INTO news_articles (src, title, url, isAI, article_date) VALUES (?, ?, ?, ?, ?)",
                       (src, title, url, isAI, article_date))
        conn.commit()
    except sqlite3.IntegrityError:
        print(f"Duplicate entry for URL: {url}")
    except Exception as err:
        print(err)

In [32]:
pd.read_sql_query("select count(*) from news_articles", conn)


,count(*)
0,51977


In [33]:
for row in merged_df.itertuples():
    # print(row)
    insert_article(cursor, row.src, row.title, row.url, row.isAI, row.date)


In [34]:
pd.read_sql_query("select count(*) from news_articles", conn)


,count(*)
0,52342


In [ ]:
# df = pd.read_sql_query("select * from news_articles", conn)
# df


In [ ]:
# merged_df = pd.read_sql_query("select * from news_articles where article_date='2024-04-'", conn)

In [35]:
len(merged_df)

365

In [36]:
merged_df = merged_df.dropna()
len(merged_df)

365

In [37]:
len(merged_df.loc[merged_df["isAI"] > 0])

156

In [38]:
len(merged_df.loc[merged_df["isAI"]])

156

In [39]:
AIdf = merged_df.loc[merged_df["isAI"] > 0].reset_index()
print(datetime.now().strftime('%H:%M:%S'), f"Found {len(AIdf)} headlines", flush=True)

14:05:41 Found 156 headlines


In [40]:
# drop spaces
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))


In [41]:
# make unique by title
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index()

In [42]:
print(datetime.now().strftime('%H:%M:%S'), f"Deduped {len(AIdf)} headlines", flush=True)

14:05:41 Deduped 152 headlines


In [43]:
# Attempt to order by topic by getting embeddings and solving a traveling salesman problem
embedding_model = 'text-embedding-3-small'
response = client.embeddings.create(input=AIdf['title'].tolist(),
                                    model=embedding_model)
embedding_list = response.data

In [44]:
embedding_df = pd.DataFrame([e.dict()['embedding'] for e in embedding_list])
embedding_df


,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,-0.003387,-0.060976,-0.024047,0.021451,0.013136,-0.023717,0.046199,0.022605,-0.018128,0.019158,...,-0.027439,0.014640,-0.001372,0.014914,0.000241,0.010417,-0.016851,-0.010341,0.002578,-0.010616
1,-0.034340,-0.037009,0.037200,0.008234,0.000415,-0.034078,-0.019541,0.045636,-0.034150,0.033602,...,-0.027501,0.046589,-0.005606,0.033983,-0.007411,0.029002,-0.006268,-0.043873,-0.020113,0.016276
2,-0.015442,0.028419,0.017353,0.005290,-0.002068,-0.020609,0.017174,0.042532,0.008908,0.000419,...,-0.034886,0.026314,-0.012694,0.011865,0.002927,0.014195,0.030674,0.002744,0.014717,-0.008721
3,0.016117,0.002881,0.041922,0.026410,0.047255,-0.042102,-0.016593,0.022790,-0.000708,0.034449,...,-0.019376,0.036047,-0.024169,0.034578,0.017830,0.023447,0.015537,-0.047178,-0.017766,0.019776
4,0.003119,-0.026173,0.007390,0.033505,-0.021488,-0.052549,-0.024238,0.011750,-0.018942,-0.010101,...,-0.005547,0.025320,0.020711,-0.001986,0.018560,-0.007033,-0.012132,0.013506,0.048577,0.008860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.028076,0.008746,-0.018431,0.034659,0.010168,-0.045541,0.004173,0.038142,-0.046956,0.013371,...,0.005587,0.037271,0.009093,0.008073,-0.016568,0.036618,0.014092,0.025722,0.072964,-0.004996
148,-0.031314,0.046351,0.033851,0.024351,-0.035833,-0.035410,0.044052,0.009044,-0.019040,0.053908,...,0.010722,0.019079,0.005411,0.011356,0.010617,0.019304,-0.010676,-0.001957,0.000633,0.003591
149,-0.013124,0.025313,0.014179,-0.029385,-0.026486,-0.024931,-0.031862,0.004806,-0.046752,0.024378,...,-0.032389,0.010304,0.008776,-0.016392,0.007379,0.043036,0.026183,-0.006058,0.001561,-0.012149
150,0.008623,0.017980,0.019145,0.037191,-0.019368,-0.025811,0.028495,0.040622,-0.018373,-0.021424,...,-0.007621,-0.001954,-0.016146,0.032555,0.019996,0.028810,0.012486,0.004371,0.014156,-0.010522


In [45]:
# naive greedy solution to traveling salesman problem

embedding_array = embedding_df.values

def nearest_neighbor_sort(embedding_array):
    # Compute the pairwise Euclidean distances between all embeddings
    distances = cdist(embedding_array, embedding_array, metric='euclidean')

    # Start from the first headline as the initial point
    path = [0]
    visited = set(path)

    while len(path) < len(embedding_array):
        last = path[-1]
        # Set the distances to already visited nodes to infinity to avoid revisiting
        distances[:, last][list(visited)] = np.inf
        # Find the nearest neighbor
        nearest = np.argmin(distances[:, last])
        path.append(nearest)
        visited.add(nearest)

    return np.array(path)

# Get the sorted indices
sorted_indices = nearest_neighbor_sort(embedding_array)

# The sorted embedding array can be obtained using these indices
sorted_embedding_array = embedding_array[sorted_indices]

sorted_indices  # Show the first few indices of the sorted path


array([  0,  85,  88,  59,  87, 150,  26,  98,  66, 143, 104,  52, 126,
       117, 118,  11,   3, 134,  39, 146,  21,  17,  93,  94, 139,  14,
       133, 125,  68,  60,  24,  58,  28,  33,  29,  27,  48, 112, 111,
       110, 137,   6,  44, 102, 101,  75, 119, 120,  72, 106, 100,  99,
        82,  73,  64, 127,  50,  46,  53, 105,   2,  81,  89,  13,  83,
        84,   1,  30, 115, 144, 145,   5,  61,  67,  49,   4, 138, 147,
        35,  62, 124,  54, 113,  18,  31,  51,   7,  71,  12,  15,  55,
        47,   9,  65, 116, 130, 141,  56,  70,  95, 122, 123, 131, 103,
        22,  23,  45, 135, 136, 129, 142,  16,  32, 108,  86,  90,  77,
        25,  40, 121, 132,  78,  79,  80, 107,  43,  74,  63, 128, 109,
        38,  19, 151,  20,  57,  96,  91,  92,  41,  42, 140,  69,  10,
       148,  76,  37,  36,  97,  34,   8, 149, 114])

In [46]:
html_str = ""
for i, j in enumerate(sorted_indices):
    row = AIdf.iloc[j]
    html_str += f'{i}.<a href="{row.url}">{row.title} - {row.src}</a><br />\n'
    display_title = row.title.replace("$", "\\\$")  # so it doesn't interpret as latex escape
    display(Markdown(f"[{i}. {display_title} - {row.src}]({row.url})"))
    

[0. 'Ask Meta AI anything': Putting Facebook's new AI to the test with a Cincinnati chili recipe - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3Lndsd3QuY29tL2FydGljbGUvbWV0YS1haS1mYWNlYm9vay1pbnN0YWdyYW0tY2luY2lubmF0aS1jaGlsaS1yZWNpcGUtY3liZXJzZWN1cml0eS82MDYwMjI4OdIBAA)

[1. Meta's AI is now in your Instagram and Facebook. Here's what it does — and whether you can turn it off - Business Insider](https://www.businessinsider.com/meta-ai-on-instagram-whatsapp-facebook-cant-turn-off-2024-4)

[2. Meta adds AI tool to Facebook, Instagram, WhatsApp, Messenger; can you turn it off? - Feedly AI](https://www.actionnewsjax.com/news/trending/meta-adds-ai-tool-facebook-instagram-whatsapp-messenger-can-you-turn-it-off/LUUEUGST5RBDDOJDJLY4VX5VJU/)

[3. Facebook's Meta AI is lying when it says you can disable it - but here's what you can do - Google News](https://news.google.com/articles/CBMic2h0dHBzOi8vd3d3LnpkbmV0LmNvbS9hcnRpY2xlL2ZhY2Vib29rcy1tZXRhLWFpLWlzLWx5aW5nLXdoZW4taXQtc2F5cy15b3UtY2FuLWRpc2FibGUtaXQtYnV0LWhlcmVzLXdoYXQteW91LWNhbi1kby_SAQA)

[4. Meta's value plummets as Zuckerberg admits AI needs more time and moneyRevenues up, but is the AI hype bubble is threatening to burst?AI + ML5 hrs|11 - The Register](https://www.theregister.com/2024/04/25/facebooks_value_plummets_as_zuckerberg/)

...

[147. Moderna Employees Should Use ChatGPT at Least 20 Times a Day, CEO Says - Feedly AI](https://gizmodo.com/moderna-ceo-chatgpt-employees-vaccines-openai-1851435620)

[148. Building an AI-Powered Summarizer using NodeJS and ApyHub API To Make Meetings Easy - HackerNoon](https://hackernoon.com/building-an-ai-powered-meeting-transcript-summarizer-using-nodejs-and-ai-api)

[149. AI- and Blood-Powered Gym Opens in New York: Continuum Club - Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vd3d3LmN1cmJlZC5jb20vYXJ0aWNsZS9jb250aW51dW0tY2x1Yi1ncmVlbndpY2gtdmlsbGFnZS1neW0tYWktYmxvb2QuaHRtbNIBAA)

[150. Writers Guild of Canada Overwhelmingly Votes to Authorize Strike Over AI, Fair Pay - Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vd3d3LmhvbGx5d29vZHJlcG9ydGVyLmNvbS9idXNpbmVzcy9idXNpbmVzcy1uZXdzL3dyaXRlcnMtZ3VpbGQtb2YtY2FuYWRhLXZvdGVzLXRvLWF1dGhvcml6ZS1zdHJpa2UtMTIzNTg4MTI0NS_SAXpodHRwczovL3d3dy5ob2xseXdvb2RyZXBvcnRlci5jb20vYnVzaW5lc3MvYnVzaW5lc3MtbmV3cy93cml0ZXJzLWd1aWxkLW9mLWNhbmFkYS12b3Rlcy10by1hdXRob3JpemUtc3RyaWtlLTEyMzU4ODEyNDUvYW1wLw)

[151. Plato's burial place finally revealed after AI deciphers ancient scroll carbonized in Mount Vesuvius eruption - Feedly AI](https://www.livescience.com/archaeology/romans/platos-burial-place-finally-revealed-after-ai-deciphers-ancient-scroll-carbonized-in-mount-vesuvius-eruption)

In [47]:
print(html_str)

0.<a href="https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3Lndsd3QuY29tL2FydGljbGUvbWV0YS1haS1mYWNlYm9vay1pbnN0YWdyYW0tY2luY2lubmF0aS1jaGlsaS1yZWNpcGUtY3liZXJzZWN1cml0eS82MDYwMjI4OdIBAA">'Ask Meta AI anything': Putting Facebook's new AI to the test with a Cincinnati chili recipe - Google News</a><br />
1.<a href="https://www.businessinsider.com/meta-ai-on-instagram-whatsapp-facebook-cant-turn-off-2024-4">Meta's AI is now in your Instagram and Facebook. Here's what it does — and whether you can turn it off - Business Insider</a><br />
2.<a href="https://www.actionnewsjax.com/news/trending/meta-adds-ai-tool-facebook-instagram-whatsapp-messenger-can-you-turn-it-off/LUUEUGST5RBDDOJDJLY4VX5VJU/">Meta adds AI tool to Facebook, Instagram, WhatsApp, Messenger; can you turn it off? - Feedly AI</a><br />
3.<a href="https://news.google.com/articles/CBMic2h0dHBzOi8vd3d3LnpkbmV0LmNvbS9hcnRpY2xlL2ZhY2Vib29rcy1tZXRhLWFpLWlzLWx5aW5nLXdoZW4taXQtc2F5cy15b3UtY2FuLWRpc2FibGUtaXQtYnV0LWhlcmVzLXdoYXQte

In [48]:
# send mail
# credentials
gmail_user = os.getenv("GMAIL_USER")
gmail_password = os.getenv("GMAIL_PASSWORD")

# Email content
from_addr = gmail_user
to_addr = 'drucev@gmail.com'
subject = f'AI news' + datetime.now().strftime('%H:%M:%S')
body = f"""
<html>
    <head></head>
    <body>
    <div>
    {html_str}
    </div>
    </body>
</html>
"""

# Setup the MIME
message = MIMEMultipart()
message['From'] = from_addr
message['To'] = to_addr
message['Subject'] = subject
message.attach(MIMEText(body, 'html'))

# Create SMTP session
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()  # Secure the connection
    server.login(gmail_user, gmail_password)
    text = message.as_string()
    server.sendmail(from_addr, to_addr, text)


# Load posts from BlueSky and format for Substack or a blog post
for now I share the interesting stuff on bluesky and then use this code to grab latest BlueSky 'tweets' and format a [Substack post](https://skynetandchill.com)

In [ ]:
client = Client(base_url='https://bsky.social')
client.login(os.environ['BSKY_USERNAME'], os.environ['BSKY_SECRET'])

mydid = {"did":"did:plc:qomkdnxrqw3gkbytdxea5z65"}

data = client.get_author_feed(
    actor=mydid['did'],
    filter='posts_and_author_threads',
    limit=50,
)


In [ ]:
def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute found URLs with an empty string
    clean_text = re.sub(url_pattern, '', text)
    return clean_text



In [ ]:
def rawfetchurl(url, timeout=60):
    """get url using requests with specified timeout. return response object, status, content-type"""
    try:
        response = requests.get(url, timeout=timeout)
    except httplib.BadStatusLine:
        print("Bad response (?) fetching url %s " % url)
        response = None
    except requests.Timeout:
        print("Timeout fetching url %s " % url)
        response = None
    except requests.ConnectionError as e:
        print("Connection error (%s) fetching url %s " % (str(e), url))
        response = None
    except requests.TooManyRedirects:
        print("Too many redirects fetching url %s " % url)
        response = None
    except requests.exceptions.MissingSchema:
        print("Missing schema url %s " % url)
        response = None
    except requests.exceptions.InvalidSchema:
        print("Invalid schema url %s " % url)
        response = None
    except requests.exceptions.InvalidURL as e:
        print("Invalid url %s, %s" % (url, str(e)))
        response = None
    except ValueError as e:
        # don't log url because possibly malformed url
        print("ValueError, url ?: ? ")
        response = None
    except httplib.IncompleteRead as e:
        print("IncompleteRead, url %s: %s " % (url, str(e)))
        response = None
    except urllib3.exceptions.SSLError as e:
        print("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ContentDecodingError as e:
        print("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ChunkedEncodingError as e:
        print("ChunkedEncodingError, url %s: %s " % (url, str(e)))
        response = None
    except UnicodeEncodeError as e:
        print("UnicodeEncodeError, url %s: %s " % (url, str(e)))
        response = None
    except OpenSSL.SSL.SysCallError as e:
        print("OpenSSL.SSL.SysCallError, url %s: %s " % (url, str(e)))
        response = -1
    except OpenSSL.SSL.ZeroReturnError as e:
        print("OpenSSL.SSL.ZeroReturnError, url %s: %s " % (url, str(e)))
        response = -1

    # except requests.packages.urllib3.exceptions.DecodeError as e:
    #     utilLog("DecodeError, url %s: %s " % (url, str(e)))
    #     response = None

    return response


imgurl = 'https://nypost.com/wp-content/uploads/sites/2/2024/02/nate-silver-calls-shut-gemini-77192719.jpg?quality=75&strip=all&w=1024'
r = rawfetchurl(imgurl)

In [ ]:

# #         impath = "%s/%d.%s" % (today_orig_dir, actualurl.id, file_ext)
# impath = 'x.jpg'
# with open(impath, 'wb') as file:
#     file.write(r.content)

# display(IPython.display.Image(filename=impath))


In [ ]:

# def resize_and_crop(image_path, output_path, size=(360, 360)):
#     # Open the image
#     image = PIL.Image.open(image_path)

#     # Calculate the aspect ratio
#     aspect_ratio = image.width / image.height
#     target_aspect_ratio = size[0] / size[1]

#     # Determine the scaling factor and new size
#     if aspect_ratio > target_aspect_ratio:
#         # Image is wider than desired aspect ratio
#         new_height = size[1]
#         new_width = int(new_height * aspect_ratio)
#     else:
#         # Image is taller than desired aspect ratio
#         new_width = size[0]
#         new_height = int(new_width / aspect_ratio)

#     # Resize the image
#     image = image.resize((new_width, new_height))

#     # Calculate coordinates to crop the image to the target size
#     left = (new_width - size[0]) / 2
#     top = (new_height - size[1]) / 2
#     right = (new_width + size[0]) / 2
#     bottom = (new_height + size[1]) / 2

#     # Crop the image
#     image = image.crop((left, top, right, bottom))

#     # Save the cropped image
#     image.save(output_path)

def resize_and_crop(input_image_path, output_image_path, desired_height=240):
    # Load the image
    with Image.open(input_image_path) as img:
        img = img.convert('RGB')

        # Calculate the new width maintaining the aspect ratio
        aspect_ratio = img.width / img.height
        new_width = int(desired_height * aspect_ratio)

        # Resize the image
        resized_img = img.resize((new_width, desired_height))

        # Save the resized image
        resized_img.save(output_image_path)

# output_path = 'square.jpg'
# resize_and_crop(impath, output_path)
# display(IPython.display.Image(filename=output_path))


In [ ]:
# attempt to remove traiing inline URLs

def truncate_last_occurrence(text: str) -> str:
    # Find the last occurrence of a space followed by any sequence of characters followed by 3 periods
    pattern = r'\s+\S+\.{3}$'
    return re.sub(pattern, '', text)

# Example text for testing

example_text = """Elon Musk says we'll run out of power capacity to run all the AI chips in 2025
newatlas.com/technology/e..."""

# Truncate the last occurrence
print(truncate_last_occurrence(example_text))



In [ ]:

# for post in data.feed:
#     post_str = post.post.record.text.rstrip()
#     post_str = truncate_last_occurrence(post_str)
#     post_url = ""
#     try:
#         post_url = post.post.record.embed.external.uri.rstrip()
#     except:
#         pass

#     print(remove_urls(post_str))
#     print(post_url)
#     print()


In [ ]:
# for i, post in enumerate(data.feed):
#     post_str = post.post.record.text.rstrip()
#     post_str = truncate_last_occurrence(post_str)
#     post_url = ""
#     tag_dict = {}
#     try:
#         post_url = post.post.record.embed.external.uri.rstrip()
#     except:
#         pass
#     if post_url:
#         tag_dict = get_og_tags(post_url)
#         display_str = f"[{post_str}]({post_url})"
#         site_name = tag_dict.get('og:site_name')
#         img_url = tag_dict.get('og:image')
#         if site_name:
#             display_str += f" - {site_name}"
#         if img_url:
#             try:
#                 r = rawfetchurl(img_url)
#                 content_type = r.headers['Content-Type']
#                 content_type = content_type[content_type.find('/')+1:]
#                 impath = f"source{i}.{content_type}"
#                 with open(impath, 'wb') as file:
#                     file.write(r.content)
#                 output_path = f'Image{i}.jpg'
#                 resize_and_crop(impath, output_path)
#                 display(IPython.display.Image(filename=output_path))
#             except Exception as e:
#                 print(e)
#         display(Markdown(display_str))
#     else:
#         display(Markdown(f"{post_str}"))


In [ ]:
imgdir = 'tmp'  # for images
delete_files(imgdir)

display(Markdown(f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"))


for i, post in enumerate(data.feed):
    post_str = post.post.record.text.rstrip()
    post_str = truncate_last_occurrence(post_str)
    post_url = ""
    tag_dict = {}
    try:
        post_url = post.post.record.embed.external.uri.rstrip()
    except:
        pass
    if post_url:
        tag_dict = get_og_tags(post_url)
        display_str = f"[{post_str}]({post_url})"
        site_name = tag_dict.get('og:site_name')
        img_url = tag_dict.get('og:image')
        if site_name:
            display_str += f" - {site_name}"

        display_str = display_str.replace("$", "\\\$")  # so Markdown doesn't interpret $ as latex escape
        if img_url:
            try:
                r = rawfetchurl(img_url)
                content_type = r.headers['Content-Type']
                content_type = content_type[content_type.find('/')+1:]
                impath = f"{imgdir}/source{i}.{content_type}"
                with open(impath, 'wb') as file:
                    file.write(r.content)
                output_path = f'{imgdir}/Image{i}.jpg'
                resize_and_crop(impath, output_path)
                display(IPython.display.Image(filename=output_path))
            except Exception as e:
                print(e)
        display(Markdown(display_str))
        display(Markdown("___"))

    else:
        display(Markdown(f"{post_str}"))

In [ ]:
data.feed[0].post.record.embed.external.thumb.dict()

In [ ]:
data.feed[0].post.record.embed.dict()

In [ ]:
print(datetime.now())

In [ ]:
# conn = sqlite3.connect('articles.db')

# c = conn.cursor()
# c.execute("delete from news_articles where article_date > '2024-04-20'")

# # Committing the changes
# conn.commit()

# # Closing the connection
# conn.close()



In [ ]:
data